In [1]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D,AveragePooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import GlobalAveragePooling2D
from keras import backend as K
from keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from keras.utils import np_utils
from imutils import paths
import numpy as np
import pickle
import cv2
import os
from keras.callbacks import ModelCheckpoint
from keras import applications
from keras import initializers
import keras

In [2]:
img_size=224
num_classes=10

INIT_LR = 0.0001
BS = 16
EPOCHS = 50

In [3]:
mobilenet=applications.mobilenet.MobileNet(input_shape=(img_size,img_size,3), alpha=1.0, depth_multiplier=1, dropout=0.1,
                                           include_top=False, weights='imagenet', input_tensor=None)


#mobilenet.summary()

#freezing all layer
#mobilenet.trainable = False

#freezing some first layers
#mobilenet.get_layer('conv1').trainable = False
#mobilenet.get_layer('conv_dw_2').trainable = False


#freezing many layers
fine_tune_at=100
for layer in mobilenet.layers[:fine_tune_at]:
  layer.trainable =  False

In [4]:
x=mobilenet.output
x=GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x)
x=Dense(128,activation='relu')(x) 
pred=Dense(num_classes, activation='softmax')(x) #final layer with softmax activation
model=Model(inputs=mobilenet.input,outputs=pred)

In [5]:
path_train='data/Face_Train'
path_test='data/Face_Test'

import os
def get_images(path):
    images = []
    labels = []
    k=0
    for subdir,_, files in os.walk(path):
        if subdir==path:
            continue
        for line in files:
            if line.startswith('.'):
                continue
            fields = subdir.strip().split('/')
            file_name=os.path.join(subdir, line)
            label=fields[-1]
            image = cv2.imread(file_name,1)
            image=cv2.resize(image,(img_size,img_size))
            images.append(image)
            labels.append(label)
        k=k+1
    return images, labels

images_train,label_train=get_images(path_train)
images_test,label_test=get_images(path_test)

In [6]:
x_train=np.array(images_train)
y_train=np.array(label_train)

x_test=np.array(images_test)
y_test=np.array(label_test)

x_train=x_train/255.0
x_test=x_test/255.0

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

path1=os.getcwd()+ '/saved_models/'
filepath=path1+"model-{epoch:02d}-{val_accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

  
H = model.fit_generator(aug.flow(x_train, y_train, batch_size=BS),
	validation_data=(x_test, y_test), steps_per_epoch=len(x_train) // BS,
	epochs=EPOCHS,callbacks=[checkpoint])


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
19/19 [==============================] - ETA: 0s - loss: 1.8227 - accuracy: 0.4110
Epoch 00001: val_accuracy improved from -inf to 0.16000, saving model to /Users/uynq/Desktop/AI_Academy/BML_Online/BML15/Codes/saved_models/model-01-0.160.hdf5
19/19 [==============================] - 22s 1s/step - loss: 1.8227 - accuracy: 0.4110 - val_loss: 2.4645 - val_accuracy: 0.1600
Epoch 2/50
19/19 [==============================] - ETA: 0s - loss: 1.1313 - accuracy: 0.6370
Epoch 00002: val_accuracy improved from 0.16000 to 0.25000, saving model to /Users/uynq/Desktop/AI_Academy/BML_Online/BML15/Codes/saved_models/model-02-0.250.hdf5
19/19 [==============================] - 22s 1s/step - loss: 1.1313 - accuracy: 0.6370 - val_loss: 2.2498 - val_accuracy: 0.2500
Epoch 3/50
19/19 [==============================] - ETA: 0s - loss: 0.7662 - accuracy: 0.7808
Epoch 00003: val_accuracy did not improve from 0.25000
19/19 